# 青空文庫から作家別に作品をダウンロードして前処理して保存

https://qiita.com/dzbt_dzbt/items/593dbd698a07c12a771c

@dzbt_dzbtさんの記事をベースに、Colab上で動作するように改変しました。有益な記事ありがとうございます。前処理の内容は記事をご覧ください。

追加内容。
- 簡易的に会話文を抜き出す機能を追加しました。
- 挿絵が入ったアーカイブの処理がエラーになる現象を回避しました。


https://www.aozora.gr.jp/

青空文庫の活動にも感謝。

### import

In [59]:
# import
import pandas as pd
from pathlib import Path
import os
import zipfile

## ダウンロードおよび保存形式の指定

In [60]:
#@title 設定
#@markdown 青空文庫の作家番号(6桁)
author_id = "000009" #@param {type:"string"}
#@markdown 青空文庫の表記での作家名
author_name = '\u30C9\u30A4\u30EB \u30A2\u30FC\u30B5\u30FC\u30FB\u30B3\u30CA\u30F3'  #@param {type:"string"}

#@markdown 2カラム目に作品名を入れるか
write_title = False #@param {type:"boolean"}

#@markdown 「」内の文章だけ抽出するか？
only_dialogue = True #@param {type:"boolean"}

#@markdown 1行目をカラム名にするか（カラム名「text」「title」)
write_header = False #@param {type:"boolean"}
#@markdown 元データをUTF-8にしたテキストファイルを保存するか
save_utf8_org = True #@param {type:"boolean"}

# 保存先
#@markdown 保存先最後の/は不要です。
SAVE_PATH = '/content' #@param {type:"string"}
out_dir = Path(SAVE_PATH + f'/out_{author_id}/')  # ファイル出力先
tx_org_dir = Path(out_dir / './org/')  # 元テキストのUTF-8変換ファイルの保存先
tx_edit_dir = Path(out_dir / './edit/')  # テキスト整形後のファイル保存先

# ダウンロード

In [61]:
# 念のため設定を確認してみます。
print('SAVE PATH:', out_dir)

SAVE PATH: /content/out_000009


In [62]:
# Colabにsubversion入ってなかったのでinstall
!apt install -y subversion

Reading package lists... Done
Building dependency tree       
Reading state information... Done
subversion is already the newest version (1.9.7-4ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [63]:
# 作家別のURLを定義
author_url = r'https://github.com/aozorabunko/aozorabunko/trunk/cards/' + author_id + '/'
os.environ['aurl'] = author_url
print('Import files from :',author_url)
# svnでダウンロード
!svn export $aurl

Import files from : https://github.com/aozorabunko/aozorabunko/trunk/cards/000009/
svn: E155000: Destination directory exists; please remove the directory or use --force to overwrite
svn: E155000: '000009' already exists


# 前処理

In [64]:
# 
import pandas as pd
from pathlib import Path
import re

def text_cleanse_df(df):
    # 本文の先頭を探す（'---…'区切りの直後から本文が始まる前提）
    head_tx = list(df[df['text'].str.contains(
        '-------------------------------------------------------')].index)
    # 本文の末尾を探す（'底本：'の直前に本文が終わる前提）
    atx = list(df[df['text'].str.contains('底本：')].index)
    if head_tx == []:
        # もし'---…'区切りが無い場合は、作家名の直後に本文が始まる前提
        head_tx = list(df[df['text'].str.contains(author_name)].index)
        head_tx_num = head_tx[0]+1
    else:
        # 2個目の'---…'区切り直後から本文が始まる
        head_tx_num = head_tx[1]+1
    df_e = df[head_tx_num:atx[0]].copy() # dfを参照せず、コピーとする。
    del df # 一応

    # 青空文庫の書式削除
    df_e = df_e.replace({'text': {'《.*?》': ''}}, regex=True)
    df_e = df_e.replace({'text': {'［.*?］': ''}}, regex=True)
    df_e = df_e.replace({'text': {'｜': ''}}, regex=True)

    # 字下げ（行頭の全角スペース）を削除
    df_e = df_e.replace({'text': {'　': ''}}, regex=True)

    # 節区切りを削除
    df_e = df_e.replace({'text': {'^.$': ''}}, regex=True)
    df_e = df_e.replace({'text': {'^―――.*$': ''}}, regex=True)
    df_e = df_e.replace({'text': {'^＊＊＊.*$': ''}}, regex=True)
    df_e = df_e.replace({'text': {'^×××.*$': ''}}, regex=True)

    # 記号、および記号削除によって残ったカッコを削除
    df_e = df_e.replace({'text': {'―': ''}}, regex=True)
    df_e = df_e.replace({'text': {'…': ''}}, regex=True)
    df_e = df_e.replace({'text': {'※': ''}}, regex=True)
    df_e = df_e.replace({'text': {'「」': ''}}, regex=True)

    # カッコ内のテキストを抽出
    if only_dialogue:
      df_e['dialogue'] = df_e['text'].str.extract(r'[「『](.+?)[」』]', expand=False)
      if write_title:
        # タイトル列を作る
        df_e['dialogue'] = df_e['text'][0]
      # column名を変更し、nanをdrop。
      df_e = df_e.drop(columns='text').rename(columns={'dialogue':'text'})
      df_e.dropna(how='any', inplace=True)

    # 一文字以下で構成されている行を削除
    df_e['length'] = df_e['text'].map(lambda x: len(x))
    df_e = df_e[df_e['length'] > 1]

    # インデックスがずれるので振りなおす
    df_e = df_e.reset_index().drop(['index'], axis=1)

    # 空白行を削除する（念のため）
    df_e = df_e[~(df_e['text'] == '')]

    # インデックスがずれるので振り直し、文字の長さの列を削除する
    df_e = df_e.reset_index().drop(['index', 'length'], axis=1)

    return df_e


def save_cleanse_text(target_file):

    try:
        # ファイルの読み込み(zipファイルにはtxtを１つしかない前提)
        print(target_file)

        temp_zip = zipfile.ZipFile(target_file)
        with zipfile.ZipFile(target_file) as zf:
          for file in zf.namelist():
            if zf.getinfo(file).filename.endswith('.txt'):
              print(file)
              # Pandas DataFrameとして読み込む（cp932で読み込まないと異体字が読めない）
              df_tmp =  pd.read_csv(temp_zip.open(file),
                                    encoding='cp932',
                                    names=['text'])
              break  

        # 元データをUTF-8に変換してテキストファイルを保存
        if save_utf8_org:
            out_org_file_nm = Path(target_file.stem + '_org_utf-8.tsv')
            df_tmp.to_csv(Path(tx_org_dir / out_org_file_nm), sep='\t',
                          encoding='utf-8', index=None)
            
        # テキスト整形
        df_tmp_e = text_cleanse_df(df_tmp)

        if write_title:
            # タイトル列を作る
            df_tmp_e['title'] = df_tmp['text'][0]

        # print(df_tmp_e.head())

        out_edit_file_nm = Path(target_file.stem + '_clns_utf-8.txt')
        df_tmp_e.to_csv(Path(tx_edit_dir / out_edit_file_nm), sep='\t',
                        encoding='utf-8', index=None, header=write_header)
    except:
      print(f'ERROR: {target_file}')


In [65]:
# main

def main():
    tx_dir = Path(str(author_id) + '/files/')
    print('Target dir:', tx_dir)
    # zipファイルのリストを作成
    zip_list = list(tx_dir.glob('*.zip'))
    # 保存ディレクトリを作成しておく
    tx_edit_dir.mkdir(exist_ok=True, parents=True)
    if save_utf8_org:
        tx_org_dir.mkdir(exist_ok=True, parents=True)

    for target_file in zip_list:
        save_cleanse_text(target_file)

main()

Target dir: 000009/files
000009/files/43471_ruby_16500.zip
09hokkyokuseigono_sencho.txt
000009/files/50712_ruby_36970.zip
kuchibiruno_nejireta_otoko.txt
000009/files/50715_ruby_36989.zip
kanja_ken_dokyonin.txt
000009/files/43498_ruby_16423.zip
the_gloria_scott.txt
000009/files/43522_ruby_16831.zip
the_empty_house.txt
000009/files/57322_ruby_57271.zip
daidaino_tane_5tsubu.txt
000009/files/54911_ruby_47219.zip
magarerumono.txt
000009/files/8_ruby_31219.zip
akage_renmei.txt
000009/files/45340_ruby_18659.zip
ango_butojinno_nazo.txt
000009/files/226_ruby_31221.zip
bohemiano_shubun.txt
000009/files/55881_ruby_50043.zip
hino_echudo.txt
000009/files/43028_ruby_15448.zip
the_yellow_face.txt
000009/files/50711_ruby_36968.zip
soenseki.txt
000009/files/43523_ruby_17190.zip
the_adventure.txt
000009/files/50716_ruby_36991.zip
gloria_scottgo.txt
000009/files/50714_ruby_36987.zip
tsuchiirono_kao.txt
000009/files/54910_ruby_47217.zip
doitsu_jiken.txt
000009/files/50718_ruby_36995.zip
hinshino_tantei.tx

# 処理後のデータをダウンロード

zipファイルにして、ダウンロード

In [66]:
from google.colab import files
# ダウンロードしたいフォルダを zip 圧縮する
!zip -r /content/aozora_files.zip $out_dir
# Colabからダウンロード
files.download('/content/aozora_files.zip')

updating: content/out_000009/ (stored 0%)
updating: content/out_000009/org/ (stored 0%)
updating: content/out_000009/org/43523_ruby_17190_org_utf-8.tsv (deflated 67%)
updating: content/out_000009/org/43498_ruby_16423_org_utf-8.tsv (deflated 66%)
updating: content/out_000009/org/54913_ruby_47223_org_utf-8.tsv (deflated 63%)
updating: content/out_000009/org/50716_ruby_36991_org_utf-8.tsv (deflated 62%)
updating: content/out_000009/org/54914_ruby_47225_org_utf-8.tsv (deflated 63%)
updating: content/out_000009/org/43497_ruby_16274_org_utf-8.tsv (deflated 67%)
updating: content/out_000009/org/50711_ruby_36968_org_utf-8.tsv (deflated 64%)
updating: content/out_000009/org/57322_ruby_57271_org_utf-8.tsv (deflated 60%)
updating: content/out_000009/org/226_ruby_31221_org_utf-8.tsv (deflated 62%)
updating: content/out_000009/org/43471_ruby_16500_org_utf-8.tsv (deflated 64%)
updating: content/out_000009/org/61393_ruby_75065_org_utf-8.tsv (deflated 62%)
updating: content/out_000009/org/42929_ruby_1

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>